# Face recognition process

## Initialization

In [ ]:
import sys
import dlib
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
# for jupyter notebook
%pylab inline
%matplotlib inline
# tell jupyter notebook what size figure should be
pylab.rcParams['figure.figsize'] = (16, 9)
# for auto-update module
%load_ext autoreload
%autoreload 2

## Load image

In [ ]:
file_name = './fig/fig1.jpeg'
image = io.imread(file_name,img_num=0).astype('uint8')

## Use HOG to detect face

In [ ]:
# HOG face detector using the built-in dlib class
face_detector = dlib.get_frontal_face_detector()

# Run the HOG face detector on the image data.
detected_faces = face_detector(image, 1)

In [ ]:
# plot the image and detected faces
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(image)
face_image = []

for i, face_rect in enumerate(detected_faces):
    print("- Facess #{} found at Left: {} Top: {} Right: {} Bottom: {}".format(i, face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()))
    # Draw a box around each face we found
    rect = patches.Rectangle((face_rect.left(),face_rect.top()),
                             face_rect.width(),
                             face_rect.height(),
                             fill=False,
                             color='red',
                             lw = 3
                            )
    ax.add_patch(rect)
    face_image.append(image[face_rect.top():face_rect.bottom(),face_rect.left():face_rect.right()])


## Posing and Projecting Faces

In [ ]:
# download the required pre-trained face detection model here:
# http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
predictor_model = "shape_predictor_68_face_landmarks.dat"
face_pose_predictor = dlib.shape_predictor(predictor_model)

# find face landmarks
def find_face_landmarks(image,face_rect):
    return face_pose_predictor(image, face_rect)

In [ ]:
# plot face landmarks
fig2 = plt.figure()
ax1 = fig2.add_subplot(121)
ax2 = fig2.add_subplot(122)
detections = []

def plotFaceWithMark(face,ax):
    ax.imshow(face)
    detect = find_face_landmarks(face, dlib.rectangle(0,0,face.shape[0],face.shape[1]))
    parts = detect.parts()
    detections.append(detect)
    ax.scatter([point.x for point in parts], [point.y for point in parts],color='blue')
    
plotFaceWithMark(face_image[0],ax1)
plotFaceWithMark(face_image[1],ax2)

## Encoding

In [ ]:
import face_recognition
import face_recognition_models

In [ ]:
face_recognition_model = face_recognition_models.face_recognition_model_location()
face_encoder = dlib.face_recognition_model_v1(face_recognition_model)
parts= detections[0].parts()
b = dlib.full_object_detection()
res = [face_encoder.compute_face_descriptor(face_image[index], detections[index]) for index in range(2)]